#Import BERT model and freeze layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -U "tensorflow-text==2.13.*"

!pip install "tf-models-official==2.13.*"

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=7d7bad24d3c93c8c5b76b0575a4a2e84a851d4d11c6e76c51e781efe27ad6ef1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_hub as hub
from transformers import BertTokenizer, TFBertModel
from keras.layers import Layer, Input, Dense
from keras.models import Model
import tensorflow_text as text
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Read excel using pandas
df = pd.read_excel('/content/drive/MyDrive/news_less_clean.xlsx')
df.shape
df.head()

,Date,URL,Title,Source,Country,LABEL
0,20240815T010000Z,https://borneobulletin.com.bn/explosions-repor...,Explosions reported near two ships off Yemen :...,borneobulletin.com.bn,Brunei,2
1,20240716T194500Z,https://www.hindustantimes.com/india-news/crew...,"Crew , including 13 Indians , still missing af...",hindustantimes.com,India,2
2,20240809T100000Z,https://www.yahoo.com/news/multiple-attacks-ta...,Multiple attacks target merchant ship off Yeme...,yahoo.com,United States,3
3,20240717T041500Z,https://timesofoman.com/article/147862-oil-tan...,Oil tanker with 13 Indians on board sinks off ...,timesofoman.com,Oman,2
4,20240812T201500Z,https://menafn.com/1108546043/Multiple-Attacks...,Multiple Attacks Target Merchant Ship Off Yemen,menafn.com,Qatar,3


In [ ]:
# Only use headline and label
df_to_split = df[['Title', 'LABEL']]
df_to_split.head()

,Title,LABEL
0,Explosions reported near two ships off Yemen :...,2
1,"Crew , including 13 Indians , still missing af...",2
2,Multiple attacks target merchant ship off Yeme...,3
3,Oil tanker with 13 Indians on board sinks off ...,2
4,Multiple Attacks Target Merchant Ship Off Yemen,3


In [ ]:
from sklearn.model_selection import train_test_split
X = df_to_split['Title']
y = df_to_split['LABEL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=424)

In [17]:
# First, let's check our labels and adjust them
from sklearn.preprocessing import LabelEncoder

# Prepare text data
MAX_WORDS = 10000  # Vocabulary size
MAX_LENGTH = 50    # Headlines are typically short

# Initialize and fit tokenizer on training data
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(X_train)

# Convert headlines to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH)

# Add early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)
# Create and fit label encoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Now modify the model with correct number of output classes
num_classes = len(label_encoder.classes_)

model = Sequential([
    Embedding(MAX_WORDS, 100, input_length=MAX_LENGTH),
    Conv1D(128, 3, activation='relu'),
    Conv1D(128, 3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')  # num_classes matches our encoded labels
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train with encoded labels
history = model.fit(
    X_train_pad,
    y_train_encoded,
    epochs=15,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# For predictions, we can convert back to original labels
predictions = model.predict(X_test_pad)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))


Epoch 1/15
15/15 [==============================] - 2s 38ms/step - loss: 2.4616 - accuracy: 0.2061 - val_loss: 2.2280 - val_accuracy: 0.2193
Epoch 2/15
15/15 [==============================] - 0s 25ms/step - loss: 2.2089 - accuracy: 0.2303 - val_loss: 2.1889 - val_accuracy: 0.2105
Epoch 3/15
15/15 [==============================] - 0s 27ms/step - loss: 2.1629 - accuracy: 0.2281 - val_loss: 2.1685 - val_accuracy: 0.2105
Epoch 4/15
15/15 [==============================] - 0s 28ms/step - loss: 2.0978 - accuracy: 0.2982 - val_loss: 2.1437 - val_accuracy: 0.2982
Epoch 5/15
15/15 [==============================] - 0s 25ms/step - loss: 2.0019 - accuracy: 0.3640 - val_loss: 2.0722 - val_accuracy: 0.3947
Epoch 6/15
15/15 [==============================] - 0s 27ms/step - loss: 1.8494 - accuracy: 0.4298 - val_loss: 1.9395 - val_accuracy: 0.3772
Epoch 7/15
15/15 [==============================] - 0s 26ms/step - loss: 1.5217 - accuracy: 0.4825 - val_loss: 1.9126 - val_accuracy: 0.4123
Epoch 8/15
15

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Get predictions
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert back to original labels
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_test_original = label_encoder.inverse_transform(y_test_encoded)

# Print detailed metrics
print("\n=== Classification Metrics ===")
print(f"Overall Accuracy: {accuracy_score(y_test_original, y_pred_labels):.4f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test_original, y_pred_labels))

# Display some sample predictions
print("\n=== Sample Predictions ===")
for i in range(5):
    print(f"\nHeadline: {X_test.iloc[i]}")
    print(f"Predicted: {y_pred_labels[i]}")
    print(f"Actual: {y_test_original[i]}")


5/5 [==============================] - 0s 5ms/step

=== Classification Metrics ===
Overall Accuracy: 0.5315

Detailed Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.44      0.53      0.48        15
           3       0.73      0.91      0.81        35
           4       0.45      0.59      0.51        22
           6       0.00      0.00      0.00         5
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         2
          11       0.12      0.11      0.12         9
          12       0.55      0.63      0.59        35
          13       0.00      0.00      0.00         5

    accuracy                           0.53       143
   macro avg       0.19      0.23      0.21       143
weighted avg       0.44      0.53      0.48    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
